https://www.tensorflow.org/tutorials/structured_data/preprocessing_layers?hl=ko

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import tensorflow as tf
%matplotlib inline

np.random.seed(3)
tf.random.set_seed(3)

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
pre = pd.read_csv('new_data_1221.csv')

pre_df = pd.DataFrame(pre)

pre_df

,person_id,gender,age_1,age_2,age_3,age_4,age_5,age_6,age_7,age_8,...,Malignant neoplastic disease,Chronic liver disease,chronic obstructive lung disease,cerebrovascular disease,chronic kidney disease,Diabetes mellitus,Ischemic heart disease,hyperlipidemia,Hypertensive disorder,cancer
0,28610148,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,39709673,0,0,0,1,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,18979243,0,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0
3,29817862,0,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0
4,26339248,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36570,12746702,0,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1
36571,65929166,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
36572,85131223,1,0,0,0,0,0,1,0,0,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1
36573,60915675,0,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [3]:
pre_df.columns = ['person_id', 'gender', 'age_1', 'age_2', 'age_3', 'age_4', 'age_5',
       'age_6', 'age_7', 'age_8', 'smoking_status_0.0', 'smoking_status_1.0',
       'smoking_status_2.0', 'smoking_status_3.0', 'bmi_1.0', 'bmi_2.0', 'bmi_3.0',
       'bmi_4.0', 'Malignant_neoplastic_disease', 'Chronic_liver_disease',
       'chronic_obstructive_lung_disease', 'cerebrovascular_disease',
       'chronic_kidney_disease', 'Diabetes_mellitus', 'Ischemic_heart_disease',
       'hyperlipidemia', 'Hypertensive_disorder', 'cancer'] 

In [4]:
pre_df.columns

Index(['person_id', 'gender', 'age_1', 'age_2', 'age_3', 'age_4', 'age_5',
       'age_6', 'age_7', 'age_8', 'smoking_status_0.0', 'smoking_status_1.0',
       'smoking_status_2.0', 'smoking_status_3.0', 'bmi_1.0', 'bmi_2.0',
       'bmi_3.0', 'bmi_4.0', 'Malignant_neoplastic_disease',
       'Chronic_liver_disease', 'chronic_obstructive_lung_disease',
       'cerebrovascular_disease', 'chronic_kidney_disease',
       'Diabetes_mellitus', 'Ischemic_heart_disease', 'hyperlipidemia',
       'Hypertensive_disorder', 'cancer'],
      dtype='object')

In [5]:
pre_df.loc[:,list(pre_df.loc[:,pre_df.isnull().any()].columns)].isnull().sum()/(len(pre_df))*100
# 결측 확인

Series([], dtype: float64)

In [6]:
pre_df = pre_df.astype('int64')

## Input Pipeline

In [7]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('cancer')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)

    return ds

### Train/ Test data split

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
train, test = train_test_split(pre_df, test_size=0.3, random_state=3)
#train, val = train_test_split(train, test_size=0.2, random_state=3)
print(len(train), 'train examples')
#print(len(val), 'validation examples')
print(len(test), 'test examples')

25602 train examples
10973 test examples


## Normalization & Metrics

In [10]:
from tensorflow.keras.layers.experimental import preprocessing

* Normalization - 데이터의 특성별 정규화입니다.
* CategoryEncoding 카테고리 인코딩 레이어입니다.
* StringLookup - 어휘의 문자열을 정수 인덱스로 매핑합니다.
* IntegerLookup - 어휘의 정수를 정수 인덱스로 매핑합니다.

In [11]:
def get_normalization_layer(name, dataset):
    # Create a Normalization layer for our feature.
    normalizer = preprocessing.Normalization()

    # Prepare a Dataset that only yields our feature.
    feature_ds = dataset.map(lambda x, y: x[name])

    # Learn the statistics of the data.
    normalizer.adapt(feature_ds)

    return normalizer

# age, bmi, egd 적용

In [12]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  # Create a StringLookup layer which will turn strings into integer indices
    if dtype == 'string':
        index = preprocessing.StringLookup(max_tokens=max_tokens)
    else:
        index = preprocessing.IntegerLookup(max_values=max_tokens)

  # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the set of possible values and assign them a fixed integer index.
    index.adapt(feature_ds)

  # Create a Discretization for our integer indices.
    encoder = preprocessing.CategoryEncoding(max_tokens=index.vocab_size())

  # Prepare a Dataset that only yields our feature.
    feature_ds = feature_ds.map(index)

  # Learn the space of possible indices.
    encoder.adapt(feature_ds)

  # Apply one-hot encoding to our indices. The lambda function captures the
  # layer so we can use them, or include them in the functional model later.
    return lambda feature: encoder(index(feature))

In [13]:
METRICS = [
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [14]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import metrics
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [15]:
def plot_metrics(history):
    metrics = ['loss', 'auc', 'precision', 'recall']
    for n, metric in enumerate(metrics):
        name = metric.replace("_"," ").capitalize()
        plt.subplot(2,2,n+1)
        plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
        plt.plot(history.epoch, history.history['val_'+metric],
                 color=colors[1], label='Val')
        plt.xlabel('Epoch')
        plt.ylabel(name)
        if metric == 'loss':
            plt.ylim([0, plt.ylim()[1]])
        elif metric == 'auc':
            plt.ylim([0.8,1])
        else:
            plt.ylim([0,1])

    plt.legend()

## Create Layer

In [16]:
from sklearn.metrics import roc_curve, auc, roc_auc_score 

In [17]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.utils import class_weight
from tensorflow.keras.utils import plot_model
from tensorflow.keras.regularizers import L1, L2, L1L2
from sklearn.utils import class_weight

In [18]:
# mini_batch
batch_size = 20
train_ds = df_to_dataset(train, batch_size=batch_size)

[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of gender:', train_features['gender'])
print('A batch of targets:', label_batch )

Every feature: ['person_id', 'gender', 'age_1', 'age_2', 'age_3', 'age_4', 'age_5', 'age_6', 'age_7', 'age_8', 'smoking_status_0.0', 'smoking_status_1.0', 'smoking_status_2.0', 'smoking_status_3.0', 'bmi_1.0', 'bmi_2.0', 'bmi_3.0', 'bmi_4.0', 'Malignant_neoplastic_disease', 'Chronic_liver_disease', 'chronic_obstructive_lung_disease', 'cerebrovascular_disease', 'chronic_kidney_disease', 'Diabetes_mellitus', 'Ischemic_heart_disease', 'hyperlipidemia', 'Hypertensive_disorder']
A batch of gender: tf.Tensor([1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1], shape=(20,), dtype=int64)
A batch of targets: tf.Tensor([0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0], shape=(20,), dtype=int64)


In [19]:
batch_size = 1024
train_ds = df_to_dataset(train, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)
all_ds = df_to_dataset(pre_df, shuffle=False, batch_size=batch_size)

In [20]:
[(_, label)] = train_ds.take(1)
y_label = np.array(label)

In [21]:
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_label),y_label)

class_weights = {0:class_weights[0], 1:class_weights[1]}

C:\ProgramData\Anaconda32\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1], y=[0 0 0 ... 0 0 1] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [22]:
#train_ds

In [23]:
all_inputs = []
encoded_features = []

# Numeric features.
'''for header in ['age','bmi','EGD']:
    numeric_col = tf.keras.Input(shape=(1,), name=header)
    normalization_layer = get_normalization_layer(header, train_ds)
    encoded_numeric_col = normalization_layer(numeric_col)
    all_inputs.append(numeric_col)
    encoded_features.append(encoded_numeric_col)'''

"for header in ['age','bmi','EGD']:\n    numeric_col = tf.keras.Input(shape=(1,), name=header)\n    normalization_layer = get_normalization_layer(header, train_ds)\n    encoded_numeric_col = normalization_layer(numeric_col)\n    all_inputs.append(numeric_col)\n    encoded_features.append(encoded_numeric_col)"

In [24]:
# Categorical features encoded as string.
categorical_cols = [ 'gender', 'age_1', 'age_2', 'age_3', 'age_4', 'age_5',
       'age_6', 'age_7', 'age_8', 'smoking_status_0.0', 'smoking_status_1.0',
       'smoking_status_2.0', 'smoking_status_3.0', 'bmi_1.0', 'bmi_2.0', 'bmi_3.0',
       'bmi_4.0', 'Malignant_neoplastic_disease', 'Chronic_liver_disease',
       'chronic_obstructive_lung_disease', 'cerebrovascular_disease',
       'chronic_kidney_disease', 'Diabetes_mellitus', 'Ischemic_heart_disease',
       'hyperlipidemia', 'Hypertensive_disorder']
for header in categorical_cols:
    print(header)
    categorical_col = tf.keras.Input(shape=(1,), name=header,dtype='int64')
    encoding_layer = get_category_encoding_layer(header, train_ds,dtype='int64')
    encoded_categorical_col = encoding_layer(categorical_col)
    all_inputs.append(categorical_col)
    encoded_features.append(encoded_categorical_col)

gender
age_1
age_2
age_3
age_4
age_5
age_6
age_7
age_8
smoking_status_0.0
smoking_status_1.0
smoking_status_2.0
smoking_status_3.0
bmi_1.0
bmi_2.0
bmi_3.0
bmi_4.0
Malignant_neoplastic_disease
Chronic_liver_disease
chronic_obstructive_lung_disease
cerebrovascular_disease
chronic_kidney_disease
Diabetes_mellitus
Ischemic_heart_disease
hyperlipidemia
Hypertensive_disorder


In [25]:
#def create_model(): #, kernel_regularizer=L2(0.001)
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(256, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(128, activation="relu")(x)
#x = tf.keras.layers.Dropout(0.05)(x)
x = tf.keras.layers.Dense(64, activation="relu")(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(32, activation="relu")(x)
#x = tf.keras.layers.Dropout(0.05)(x)
x = tf.keras.layers.Dense(16, activation="relu")(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(8, activation="relu")(x)
#x = tf.keras.layers.Dropout(0.1)(x)
#x = tf.keras.layers.Dense(8, activation="relu")(x)
#x = tf.keras.layers.Dropout(0.1)(x)
#x = tf.keras.layers.Dense(8, activation="relu")(x)
#x = tf.keras.layers.Dropout(0.1)(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(all_inputs, output)
model.compile(optimizer='rmsprop',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=METRICS)

In [26]:
#tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR", to_file='./model1.png')

In [27]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

In [28]:
history = model.fit(train_ds, epochs=100, validation_data=test_ds,callbacks=[early_stopping],class_weight = class_weights)

Epoch 1/100


C:\Users\dladu\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\functional.py:588: UserWarning: Input dict contained keys ['person_id'] which did not match any model input. They will be ignored by the model.
  warnings.warn(
C:\Users\dladu\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\backend.py:5016: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


26/26 [==============================] - 10s 178ms/step - loss: 0.4279 - recall: 0.8134 - precision: 0.2771 - tp: 1996.0000 - fp: 5206.0000 - tn: 17942.0000 - fn: 458.0000 - accuracy: 0.7788 - auc: 0.8827 - prc: 0.4466 - val_loss: 0.3713 - val_recall: 1.0000 - val_precision: 0.3501 - val_tp: 1059.0000 - val_fp: 1966.0000 - val_tn: 7948.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.8208 - val_auc: 0.9656 - val_prc: 0.7695
Epoch 2/100
26/26 [==============================] - 1s 29ms/step - loss: 0.2406 - recall: 0.9560 - precision: 0.3633 - tp: 2346.0000 - fp: 4111.0000 - tn: 19037.0000 - fn: 108.0000 - accuracy: 0.8352 - auc: 0.9572 - prc: 0.7303 - val_loss: 0.2882 - val_recall: 1.0000 - val_precision: 0.3510 - val_tp: 1059.0000 - val_fp: 1958.0000 - val_tn: 7956.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.8216 - val_auc: 0.9685 - val_prc: 0.8133
Epoch 3/100
26/26 [==============================] - 1s 41ms/step - loss: 0.2106 - recall: 0.9698 - precision: 0.3733 - tp: 2380.0000 - fp: 3996

In [31]:
# model.save('gbd_0108') ㅐ모델 저장 코드

In [28]:
#plot_metrics(history)

In [31]:
config = model.get_config()

In [32]:
config #모델 구성이 포함된 Python dict가 반환

# Sequential.from_config(config)(Sequential 모델) 또는 
# Model.from_config(config)(Functional API 모델)를 통해 동일한 모델을 재구성할 수 있습니다

{'name': 'model',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 1),
    'dtype': 'int64',
    'sparse': False,
    'ragged': False,
    'name': 'gender'},
   'name': 'gender',
   'inbound_nodes': []},
  {'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 1),
    'dtype': 'int64',
    'sparse': False,
    'ragged': False,
    'name': 'age_1'},
   'name': 'age_1',
   'inbound_nodes': []},
  {'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 1),
    'dtype': 'int64',
    'sparse': False,
    'ragged': False,
    'name': 'age_2'},
   'name': 'age_2',
   'inbound_nodes': []},
  {'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 1),
    'dtype': 'int64',
    'sparse': False,
    'ragged': False,
    'name': 'age_3'},
   'name': 'age_3',
   'inbound_nodes': []},
  {'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 1),
    'dtype': 'int64',
    'sparse': False,
    'ragged': Fals

In [29]:
testing = df_to_dataset(test, batch_size=len(test),shuffle=False)

mets = model.evaluate(testing)

#print("Loss", mets[0])
print("Accuracy", mets[5])
print("AUC", mets[8])

In [30]:
results = model.evaluate(testing)
for name, value in zip(model.metrics_names, results):
    print(name, ': ', value)

1/1 [==============================] - 0s 273ms/step - loss: 0.2013 - recall: 0.9367 - precision: 0.4368 - tp: 992.0000 - fp: 1279.0000 - tn: 8635.0000 - fn: 67.0000 - accuracy: 0.8773 - auc: 0.9696 - prc: 0.8190
loss :  0.20129528641700745
recall :  0.9367327690124512
precision :  0.43681198358535767
tp :  992.0
fp :  1279.0
tn :  8635.0
fn :  67.0
accuracy :  0.877335250377655
auc :  0.9695984721183777
prc :  0.8190070390701294


In [31]:
pred = model.predict(testing)
pred = [ 1 if x >= 0.5 else 0 for x in pred]


[(_, label)] = testing.take(1)
print(confusion_matrix(label,pred,labels=[1,0]))
print(classification_report(label,pred))

[[ 992   67]
 [1279 8635]]
              precision    recall  f1-score   support

           0       0.99      0.87      0.93      9914
           1       0.44      0.94      0.60      1059

    accuracy                           0.88     10973
   macro avg       0.71      0.90      0.76     10973
weighted avg       0.94      0.88      0.90     10973

